# Examples

This jupyter notebook contains examples of running the various functions available in _archeryutils_.
Where different options exist for the examples they are listed. Users are encouraged to re-run various cells of this notebook with different options to explore the full functionality of the code.

## 0. Getting set up

To start off using archeryutils we need to import it.
Assuming it has been installed into the local environment according to the repository documentation this can be done as follows:

In [27]:
import archeryutils as au

## 1. Basic Building Blocks
The basic building blocks of the archeryutils package are the `targets.Target` and the `rounds.Round` class.

### Target

A target is defined with the following attributes:
- Face diameter [m]
- Scoring system
- Distance [default m]

The following scoring systems are possible:
- `"5_zone"`,
- `"10_zone"`,
- `"10_zone_compound"`,
- `"10_zone_6_ring"`,
- `"10_zone_5_ring"`,
- `"10_zone_5_ring_compound"`,
- `"WA_field"`,
- `"IFAA_field"`,
- `"IFAA_field_expert"`,
- `"Beiter_hit_miss"`,
- `"Worcester"`,
- `"Worcester_2_ring"`

where an `"_n_ring"` suffix indicates a reduced scoring area with only the central n rings, and the `"_compound"` suffix indicates the system where only the x-ring scores 10 points.

So we could define the target shot on the WA 720 70m round as:

In [28]:
my720target = au.targets.Target(1.22, "10_zone", 70.0)

and the corresponding target shot by compounds (reduced 80cm face at 50m) as:

In [29]:
mycompound720target = au.targets.Target(0.80, "10_zone_6_ring", 50.0)

The target object can also take the optional arguments of:
- `native_dist_unit` - string - default = `"metres"`
- `indoor` - boolean - default = `False`

to provide an imperial distance in yards and indicate if the round is to be shot indoors (where rules may be different).

For example, the longest target on an IFAA field rounds could be defined as follows:

In [30]:
myIFAATarget = au.targets.Target(0.80, "IFAA_field", 80.0, native_dist_unit="yards")

and the target shot for the Archery GB Postsmouth round as:

In [31]:
myPortsmouthTarget = au.targets.Target(
    0.60, "10_zone", 20.0, native_dist_unit="yards", indoor=True
)

Target objects have the ability to return the maximum possible score from the type of face specified through the `max_score` method:

In [32]:
for target in [my720target, mycompound720target, myIFAATarget, myPortsmouthTarget]:
    print(target.max_score())

10.0
10.0
5.0
10.0


### Pass
The natural extension to the `targets.Target` class is to shoot a number of arrows at it.
in _archeryutils_ this is called a "pass" and is defined using the `rounds.Pass` class which wraps around the `targets.Target` class.

This takes a number of arrows followed by all of the arguments to target defined above.

For example, to define the 36 arrow pass that forms the first distance on a WA 1440 70m round, or the first half of the WA 720 70m round we use:

In [33]:
my70mPass = au.rounds.Pass(36, 1.22, "10_zone", 70.0)

Like the `targets.Target` class a `rounds.Pass` also has a `max_score()` method, but this now returns the maximum possible score for the pass (`n_arrows * Target.max_score()`):

In [34]:
print(my70mPass.max_score())

360.0


### Round
In reality we rarely use the `targets.Target` or `rounds.Pass` objects by themselves, however, instead preferring to use the `rounds.Round` class. This defines multiple passes to form what is commonly known as a round.

A `rounds.Round` object is defined with a string `name` to provide a popular name for the round and a list of `rounds.Pass` objects.

It may also take the following optional string arguments:
- `location` - where the round is shot, e.g. 'Indoor', 'Outdoor', 'Field' etc.
- `body` - The governing body the round is defined by, e.g. 'WA', 'IFAA', 'AGB', 'AA' etc.
- `family` - The larger family of rounds to which this round belongs, e.g. 'wa_1440', 'wa_720', 'national' etc.

So to define a WA 720 70m round we can re-use our variable `my70mPass` from above as follows:

In [35]:
my720Round = au.rounds.Round(
    "WA 720 (70m)",
    [my70mPass, my70mPass],
    location="Outdoor Target",
    body="WA",
    family="WA720",
)

Again we have a method for maximum score:

In [36]:
print(my720Round.max_score())

720.0


### Default Rounds

A number of useful rounds are pre-defined and come preloaded as dictionaries that can be imported:

In [37]:
from archeryutils import load_rounds

agb_outdoor = load_rounds.AGB_outdoor_imperial

for round_i in agb_outdoor.values():
    print(round_i.name)

York
Hereford
Bristol I
Bristol II
Bristol III
Bristol IV
Bristol V
St. George
Albion
Windsor
Windsor 50
Windsor 40
Windsor 30
New Western
Long Western
Western
Western 50
Western 40
Western 30
American
St Nicholas
New National
Long National
National
National 50
National 40
National 30
New Warwick
Long Warwick
Warwick
Warwick 50
Warwick 40
Warwick 30


The individial rounds are accessible via 'dot' notation (using the alias listed in `agb_outdoor.keys()`) as follows:

In [38]:
agb_outdoor.york.get_info()

agb_outdoor.york.max_score()

A York consists of 3 passes:
	- 72 arrows at a 122.0 cm target at 100.0 yards.
	- 48 arrows at a 122.0 cm target at 80.0 yards.
	- 24 arrows at a 122.0 cm target at 60.0 yards.


1296.0

Possible options for round collections are:
- `AGB_outdoor_imperial` - Archery GB outdoor imperial rounds
- `AGB_outdoor_metric` - Archery GB outdoor metric rounds
- `AGB_indoor` - Archery GB indoor rounds
- `WA_outdoor` - World Archery outdoor rounds
- `WA_indoor` - World Archery indoor rounds
- `WA_field` - World Archery field rounds
- `IFAA_field` - IFAA indoor and outdoor rounds
- `AGB_VI` - Archery GB Visually Impaired rounds
- `WA_VI` - World Archery Visually Impaired rounds
- `custom` - custom rounds such as individual distances, 252 awards, frostbites etc.

## 2. Handicap Schemes
_archeryutils_ provides functionality for calculating various [handicaps/skill ratings](https://jackatkinson.net/post/archery_handicap/) from scores. These include both the popular Archery GB and Archery Australia schemes.

To use these functionalities import the handicap equations and functions modules as below.

It will also help to instantiate a `HcParams` object that contains key parameters for the handicap schemes.
This comes as a preloaded dataclass, though the values of the variables can be changed or loaded from a `.json` file. For an explanation of the different values consult the [class definition in `archeryutils/handicaps/handicap_equations.py`](https://github.com/jatkinson1000/archeryutils/blob/main/archeryutils/handicaps/handicap_equations.py).

In [39]:
from archeryutils import handicap_equations as hc_eq
from archeryutils import handicap_functions as hc_func

hcparams = hc_eq.HcParams()
print(hcparams)

HcParams(AGB_datum=6.0, AGB_step=3.5, AGB_ang_0=0.0005, AGB_kd=0.00365, AGBo_datum=12.9, AGBo_step=3.6, AGBo_ang_0=0.0005, AGBo_k1=1.429e-06, AGBo_k2=1.07, AGBo_k3=4.3, AGBo_p1=2.0, AGBo_arw_d=0.00714, AA_k0=2.37, AA_ks=0.027, AA_kd=0.004, AA2_k0=2.57, AA2_ks=0.027, AA2_f1=0.815, AA2_f2=0.185, AA2_d0=50.0, AA_arw_d_out=0.005, arw_d_in=0.0093, arw_d_out=0.0055)


### Score from handicap

It is then possible to use the `score_for_round` function to calculate score on any `rounds.Round` for a given handicap/skill rating.

This requires a round, handicap/skill rating, scheme, and set of handicap parameters.

Possible options for the scheme are:
- `"AGB"` - The 2023 Archery GB handicap system developed by Jack Atkinson
- `"AGBold"` - The old Archery GB handicap system developed by David Lane
- `"AA2"` - The 2014 Archery Australia Skill rating system developed by Jim Park
- `"AA"` - The old Archery Australia skill rating system developed by Jim Park

For example, to calculate the score on a York round for a handicap of 38 using the 2023 Archery GB scheme we run:

In [40]:
score_from_hc, _ = hc_eq.score_for_round(
    agb_outdoor.york,
    38,
    "AGB",
    hcparams,
)

print(f"A handicap of 38 on a York is a score of {score_from_hc}.")

A handicap of 38 on a York is a score of 940.0.


Note that it is possible to obtain scores for decimal handicaps:

In [41]:
score_from_hc, _ = hc_eq.score_for_round(
    agb_outdoor.york,
    38.25,
    "AGB",
    hcparams,
)

print(f"A handicap of 38.25 on a York is a score of {score_from_hc}.")

A handicap of 38.25 on a York is a score of 936.0.


By default this function returns a round score as would appear in handicap tables and is physically attainable when shooting a round. The rounding mechanism (round/floor/ceil) varies by scheme. However, it is possible to return the mathematically continuous score by setting the `round_score_up` optional argument to be `False`:

In [42]:
score_from_hc, _ = hc_eq.score_for_round(
    agb_outdoor.york,
    38.25,
    "AGB",
    hcparams,
    round_score_up=False,
)

print(f"A handicap of 38.25 on a York is a decimal score of {score_from_hc}.")

A handicap of 38.25 on a York is a decimal score of 935.1392717845214.


### Handicap from Score

Mathematically is is easy to define a score for a given handicap, but often the opposite is required, where one wishes to obtain the handicap given a score.

To perform this operation use the `handicap_from_score()` function which takes a score, round, handicap scheme, and set of parameters.
By default it returns the decimal handicap corresponding to the provided score exactly.
However, it is possible to return the integer handicap value that the score would correspond to in a handicap table by setting the `int_prec` optional argument to `True`. Remember that the rounding mechanism (round/floor/ceil) varies by scheme.


For example, to get the 2023 Archery GB handicap given by a score of 950 on a York round:

In [43]:
hc_from_score = hc_func.handicap_from_score(
    950,
    agb_outdoor.york,
    "AGB",
    hcparams,
)
print(f"A score of 950 on a York is a continuous handicap of {hc_from_score}.")

hc_from_score = hc_func.handicap_from_score(
    950,
    agb_outdoor.york,
    "AGB",
    hcparams,
    int_prec=True,
)
print(f"A score of 950 on a York is a discrete handicap of {hc_from_score}.")

A score of 950 on a York is a continuous handicap of 37.374134403932686.
A score of 950 on a York is a discrete handicap of 38.0.


### Handicap Tables

A further functionality of the 

In [44]:
import numpy as np

handicaps = np.arange(0.0, 151.0, 1.0)
rounds = [
    agb_outdoor.york,
    agb_outdoor.hereford,
    agb_outdoor.albion,
    agb_outdoor.windsor,
]
# The following allows printing of handicap tables for an entire group of rounds:
# rounds = list(load_rounds.AGB_outdoor_imperial.values())

hc_func.print_handicap_table(
    handicaps,
    "AGB",
    rounds,
    hcparams,
)

      Handicap          York      Hereford        Albion       Windsor
    0.00000000 1284.00000000 1295.00000000  972.00000000  972.00000000
    1.00000000 1281.00000000 1294.00000000  971.00000000  972.00000000
    2.00000000 1279.00000000 1294.00000000  971.00000000  972.00000000
    3.00000000 1276.00000000 1293.00000000  971.00000000  972.00000000
    4.00000000 1272.00000000 1292.00000000  970.00000000  972.00000000
    5.00000000 1268.00000000 1290.00000000  969.00000000  972.00000000
    6.00000000 1264.00000000 1289.00000000  969.00000000  972.00000000
    7.00000000 1260.00000000 1287.00000000  968.00000000  972.00000000
    8.00000000 1255.00000000 1285.00000000  967.00000000  972.00000000
    9.00000000 1250.00000000 1283.00000000  966.00000000  972.00000000
   10.00000000 1245.00000000 1281.00000000  964.00000000  972.00000000
   11.00000000 1239.00000000 1278.00000000  963.00000000  971.00000000
   12.00000000 1233.00000000 1275.00000000  961.00000000  971.00000000
   13.

The following optional arguments can be passed to `print_handicap_table()`:
- `round_scores_up` - round scores to discrete values as appropriate for the scheme
- `clean_gaps` - if `True` duplicate scores will be displayed for the first occurrence only
- `filename` - if provided as a string the table will be saved to file
- `int_prec` - if `True` then values will be printed as integers rather than decimals

It is also possible to pass an array of non-integer handicaps.

The effect of these variables can be examined by changing their values in the following:

In [45]:
handicaps = np.arange(0.0, 51.0, 0.5)


hc_func.print_handicap_table(
    handicaps,
    "AGB",
    rounds,
    hcparams,
    round_scores_up=False,
    clean_gaps=False,
    filename="test_handicap_table.txt",
    int_prec=False,
)

      Handicap          York      Hereford        Albion       Windsor
    0.00000000 1283.43539112 1294.41128340  971.20310600  971.99244624
    0.50000000 1282.23101439 1294.14732559  971.07019646  971.98968696
    1.00000000 1280.95163666 1293.85033513  970.92047684  971.98606417
    1.50000000 1279.59589344 1293.51777056  970.75259933  971.98135501
    2.00000000 1278.16257099 1293.14707366  970.56519185  971.97529244
    2.50000000 1276.65060131 1292.73568446  970.35686337  971.96755961
    3.00000000 1275.05905507 1292.28105576  970.12620874  971.95778415
    3.50000000 1273.38713286 1291.78066714  969.87181326  971.94553236
    4.00000000 1271.63415490 1291.23203803  969.59225663  971.93030358
    4.50000000 1269.79954959 1290.63273977  969.28611639  971.91152478
    5.00000000 1267.88284129 1289.98040647  968.95197074  971.88854538
    5.50000000 1265.88363751 1289.27274454  968.58840071  971.86063266
    6.00000000 1263.80161600 1288.50754076  968.19399174  971.82696765
    6.

## 3. Classifications

As well as handicap functionalities _archeryutils_ fontains functionalities for calculating Archery GB classifications.
These are accessed by importing the `classifications` module:

In [46]:
from archeryutils import classifications as class_func

### Classification from score

To get a classification that results from a score use the `calculate_X_classification()` function, where `X` corresponds to the classification scheme being used (`AGB_outdoor`, `AGB_indoor`, `AGB_field`).

This takes following arguments:
- a string of a round alias (see [Default Rounds](#Default-Rounds)).
- a score
- a string of bowstyle (`"compound"`, `"recurve"`, `"longbow"`, `"barebow"`, `"traditional"`, `"flatbow"`)
- a string of gender under Archery GB (`"male"` or `"female"`)
- an Archery GB age group (`"50+"`, `"adult"`, `"under 21"`, `"under 18"`, etc.)

and returns a string corresponding to the classification it obtains.

These can be investigated in the following code snippet which uses a number of examples:

In [47]:
# AGB Outdoor
class_from_score = class_func.calculate_AGB_outdoor_classification(
    "hereford",
    965,
    "recurve",
    "male",
    "50+",
)
print(
    f"A score of 965 on a Hereford is class {class_from_score} for a 50+ male recurve."
)

A score of 965 on a Hereford is class B2 for a 50+ male recurve.


In [48]:
# AGB Indoor
class_from_score = class_func.calculate_AGB_indoor_classification(
    "wa18",
    562,
    "compound",
    "female",
    "adult",
)
print(
    f"A score of 562 on a WA 18 is class {class_from_score} for adult female compound."
)

A score of 562 on a WA 18 is class C for adult female compound.


In [49]:
# AGB Field
class_from_score = class_func.calculate_AGB_field_classification(
    "wa_field_24_blue_unmarked",
    168,
    "traditional",
    "male",
    "under 18",
)
print(
    f"A score of 168 on a WA Unmarked 24 is class {class_from_score} for an under 18 male traditional."
)

A score of 168 on a WA Unmarked 24 is class 1st Class for an under 18 male traditional.


### Classification scores

As well as generating a classification from a score there is also the inverse functionality of obtaining scores required for classifications. This can be done using the `X_classification_scores()` functions.

These take a round alias and categories as strings above, and return a list of scores required for each classification in descending order.

Where a classification is not available from a particular round a fill value of -9999 is returned.

In [50]:
class_scores = class_func.AGB_outdoor_classification_scores(
    "hereford",
    "recurve",
    "male",
    "adult",
)
print(class_scores)

[-9999, -9999, -9999, -9999, 985, 863, 721, 571, 427]


In [51]:
class_scores = class_func.AGB_indoor_classification_scores(
    "portsmouth",
    "compound",
    "female",
    "adult",
)
print(class_scores)

[594, 586, 562, 518, 453, 349, 207, 160]


In [52]:
class_scores = class_func.AGB_field_classification_scores(
    "wa_field_24_blue_marked",
    "flatbow",
    "female",
    "under 18",
)
print(class_scores)

[158, 147, 134, 121, 107, 95]
